In [1]:
import torch
import torchvision
import sys
sys.path.append('..')
import failure_directions
import numpy as np
import torchvision.transforms as transforms
from torch.cuda.amp import autocast
from tqdm import tqdm
import torch.nn as nn
import matplotlib.pyplot as plt


/mnt/cfs/home/saachij/conda_envs/ffcv_cfs/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Training the base model

First, we're going to train a CIFAR-10 model. We just put an example setup here (make sure to evaluate these cells, but feel free to skip to the next section for our method!)

### Setting up the dataset
Ok, first let's set up the dataset. We're going to use CIFAR-10, so let's download it below.

We are going to train on 20% of the training dataset, and will let another 20% be validation. 

In [2]:
hparams = {
    'mean': [125.307, 122.961, 113.8575], 'std': [51.5865, 50.847, 51.255],
    'num_classes': 10, 'arch': 'resnet18', 'arch_type': 'cifar_resnet', 'batch_size': 512,
    'training': {
        'epochs': 35, 'lr': 0.5,
        'optimizer': {'momentum': 0.9, 'weight_decay': 5.0E-4},
        'lr_scheduler':{'type': 'cyclic', 'lr_peak_epoch': 5}
    }
}

fill_color = tuple(map(int, hparams['mean']))

base_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=np.array(hparams['mean'])/255, std=np.array(hparams['std'])/255)])

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    base_transform
])

# For visualization
INV_NORM = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [255/x for x in hparams['std']]),
                                transforms.Normalize(mean = [-x /255 for x in hparams['mean']],
                                                     std = [ 1., 1., 1. ])])
TOIMAGE = transforms.Compose([INV_NORM, transforms.ToPILImage()])

In [3]:
ds_root = "/mnt/cfs/datasets/cifar"
orig_train_ds = torchvision.datasets.CIFAR10(ds_root, train=True, transform=base_transform)
aug_train_ds = torchvision.datasets.CIFAR10(ds_root, train=True, transform=train_transform)
test_ds = torchvision.datasets.CIFAR10(ds_root, train=False, transform=base_transform)

all_train_indices = torch.arange(len(orig_train_ds))
val_indices = all_train_indices[::5]
train_indices = all_train_indices[1::5]

train_ds = torch.utils.data.Subset(aug_train_ds, train_indices)
val_ds = torch.utils.data.Subset(orig_train_ds, val_indices)
no_aug_train_ds = torch.utils.data.Subset(orig_train_ds, train_indices)

In [4]:
bsz = hparams['batch_size']
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=bsz, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=bsz, shuffle=False, drop_last=False)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=bsz, shuffle=False, drop_last=False)
no_aug_train_loader = torch.utils.data.DataLoader(no_aug_train_ds, batch_size=bsz, shuffle=False, drop_last=False)

loaders = {'train': no_aug_train_loader, 'test': test_loader, 'val': val_loader} 

### Training a Model
Ok, let's train a model. We'll train a ResNet-18. 

we have a pre-trained model [here](https://www.dropbox.com/s/1gtlmbe4k2dzh9w/example_ckpt.pt?dl=0) if you don't want to wait for training

In [5]:
# if you've already trained the model just load it here
build_fn = failure_directions.model_utils.BUILD_FUNCTIONS[hparams['arch_type']]
path = "example_ckpt.pt"
model = failure_directions.model_utils.load_model(path, build_fn)
model = model.cuda()


{'training_args': {'epochs': 35, 'lr': 0.5, 'optimizer': {'momentum': 0.9, 'weight_decay': 0.0005}, 'lr_scheduler': {'type': 'cyclic', 'lr_peak_epoch': 5}, 'iters_per_epoch': 19}, 'epoch': 34, 'training_metrics': {'loss': 0.29991454199740764, 'acc': 0.8988486842105263}, 'val_metrics': {'loss': 0.7290558261871338, 'acc': 0.7687000005722046}}


otherwise, train it below

In [22]:
build_fn = failure_directions.model_utils.BUILD_FUNCTIONS[hparams['arch_type']]
model = build_fn(hparams['arch'], hparams['num_classes'])
model = model.cuda()

training_args=hparams['training']
training_args['iters_per_epoch'] = len(train_loader)
trainer = failure_directions.LightWeightTrainer(training_args=hparams['training'],
                                                exp_name='temp', enable_logging=True,
                                                bce=False, set_device=True)
trainer.fit(model, train_loader, val_loader)

logging in  runs/temp/version_0


Val Epoch: 0: 100%|██████████| 20/20 [00:03<00:00,  5.71it/s, loss=1.33e+3, acc=0.0846]


LR: 0.09999999999999999, Train Loss: 2.6122, Train Acc: 0.1816, Val Loss: 1494.3194, Val Acc: 0.0977


Val Epoch: 1: 100%|██████████| 20/20 [00:03<00:00,  5.77it/s, loss=308, acc=0.11]  


LR: 0.19999999999999998, Train Loss: 2.4504, Train Acc: 0.1882, Val Loss: 324.7413, Val Acc: 0.1040


...

LR: 0.033333333333333326, Train Loss: 0.3746, Train Acc: 0.8699, Val Loss: 0.7592, Val Acc: 0.7544


Val Epoch: 33: 100%|██████████| 20/20 [00:03<00:00,  5.95it/s, loss=0.727, acc=0.757]


LR: 0.016666666666666663, Train Loss: 0.3329, Train Acc: 0.8844, Val Loss: 0.7372, Val Acc: 0.7672


Val Epoch: 34: 100%|██████████| 20/20 [00:03<00:00,  5.91it/s, loss=0.736, acc=0.772]


LR: 0.0, Train Loss: 0.2999, Train Acc: 0.8988, Val Loss: 0.7291, Val Acc: 0.7687


now let's evaluate the model and get the predictions!

In [6]:
def evaluate_model(loader):
    with torch.no_grad():
        with autocast():
            gts, preds, confs = [], [], []
            for x, y in tqdm(loader):
                x = x.cuda()
                logits = model(x)
                gts.append(y.cpu())
                preds.append(logits.argmax(-1).cpu())
                softmax_logits = nn.Softmax(dim=-1)(logits)
                confs.append(softmax_logits[torch.arange(logits.shape[0]), y].cpu())
    gts = torch.cat(gts).numpy()
    preds = torch.cat(preds).numpy()
    confs = torch.cat(confs).numpy()
    return gts, preds, confs

In [7]:
model = model.eval()
run_dict = {}
for split, loader in loaders.items():
    run_dict[split] = evaluate_model(loader) # gts, preds, confs

100%|██████████| 20/20 [00:03<00:00,  6.24it/s]


# Bringing in CLIP

In [8]:
clip_processor = failure_directions.CLIPProcessor(ds_mean=hparams['mean'], ds_std=hparams['std'])

In [9]:
clip_features = {}
for split, loader in loaders.items():
    clip_features[split] = clip_processor.evaluate_clip_images(loader)

100%|██████████| 20/20 [00:04<00:00,  4.49it/s]


# Fitting the SVM

In [10]:
svm_fitter = failure_directions.SVMFitter()
svm_fitter.set_preprocess(clip_features['train'])
val_gts, val_preds, _ = run_dict['val']
cv_scores = svm_fitter.fit(preds=val_preds, ys=val_gts, latents=clip_features['val'].numpy())

updating whitening


100%|██████████| 10/10 [01:05<00:00,  6.57s/it]


In [11]:
svm_predictions = {}
svm_decision_values = {}
for split, loader in loaders.items():
    gts_, _, _ = run_dict[split]
    mask, dv = svm_fitter.predict(ys=gts_, latents=clip_features[split], compute_metrics=False)
    svm_predictions[split] = mask
    svm_decision_values[split] = dv

100%|██████████| 10/10 [00:07<00:00,  1.36it/s]


## Captioning our failure modes!

In [12]:
from failure_directions.src.clip_utils import get_caption_set

In [13]:
captions = failure_directions.get_caption_set('CIFAR')

excluding plott hound
excluding vizsla
excluding norwich terrier
excluding rottweiler
excluding belgian sheepdog
excluding soft-coated wheaten terrier
excluding pekinese
excluding blenheim spaniel
excluding german short-haired pointer
excluding malamute
excluding liver-spotted dalmatian
excluding ibizan hound
excluding black-and-tan coonhound
excluding black-and-tan coonhound
excluding rhodesian ridgeback
excluding brabancon griffon
excluding old english sheepdog
excluding curly-coated retriever
excluding appenzeller
excluding bullterrier
excluding coonhound
excluding seizure-alert dog
excluding clumber
excluding dandie dinmont
excluding bedlington terrier
excluding housedog
excluding leonberg
excluding american staffordshire terrier
excluding weimaraner
excluding doberman
excluding staffordshire bullterrier
excluding staffordshire bullterrier
excluding bouvier des flandres
excluding bouvier des flandres
excluding bouvier des flandres
excluding smooth-haired fox terrier
excluding senne

  0%|          | 0/11 [00:00<?, ?it/s]

dog
bird


 18%|█▊        | 2/11 [00:00<00:00, 12.98it/s]

automobile
airplane
truck
ship
frog
cat
horse
deer
reference


100%|██████████| 11/11 [00:00<00:00, 50.66it/s]


In [ ]:
selected_captions = []
for target_c in range(10):
    target_c_name = test_ds.classes[target_c]
    caption_set = captions[target_c_name]['all']
    reference = captions['reference'][target_c]
    print(target_c_name, reference)
    decisions, _ = clip_processor.get_caption_scores(captions=caption_set,
                                                     reference_caption=reference,
                                                     svm_fitter=svm_fitter,
                                                     target_c=target_c)
    selected_captions.append((
        caption_set[np.argmin(decisions)],
        caption_set[np.argmax(decisions)], decisions))


airplane a photo of a airplane


100%|██████████| 10/10 [00:01<00:00,  5.52it/s]


automobile a photo of a automobile


100%|██████████| 10/10 [00:01<00:00,  7.06it/s]


bird a photo of a bird


  0%|          | 0/10 [00:00<?, ?it/s]

## Visualize!

In [ ]:

def visualize_images(ds, ds_indices, ds_values, K=10, title=""):
    fig, ax = plt.subplots(1, K, figsize=(K*2, 3))
    for i in range(K):
        idx = ds_indices[i]
        ax[i].imshow(TOIMAGE(ds[idx][0]))
        ax[i].axis(False)
        ax[i].set_title(f"{ds_values[i]:0.3f}")
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

gts = run_dict['test'][0]
for c in range(10):
    print("---")
    selected_caps = selected_captions[c]
    mask = gts == c
    masked_indices = np.arange(len(mask))[mask]
    dv = svm_decision_values['test'][mask]
    bottom_dv = np.argsort(dv)
    top_dv = bottom_dv[::-1]
    for order_name, order, cap in (
        ('bottom', bottom_dv, selected_caps[0]),
        ("top", top_dv, selected_caps[1]),
    ):
        vals = dv[order]
        visualize_images(test_ds, masked_indices[order], vals, title=cap)
        print("\n")
    